# Install libraries

In [67]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import folium
import geopandas as gpd
import json
from folium import Choropleth

## General Settings

In [17]:
# Ensure pandas shows all columns in dataframe
pd.set_option('display.max_columns', None)

In [18]:
#Set default decimal point to not be scientific notation
pd.set_option('display.float_format', lambda x: '%.0f' % x)

## Import Datasets

In [6]:
city_geo = r'/Users/quincy/Documents/CF_Da/6/Airbnb Berlin/03 Scripts/berlin_bezirke.geojson'

In [7]:
city_geo

'/Users/quincy/Documents/CF_Da/6/Airbnb Berlin/03 Scripts/berlin_bezirke.geojson'

In [15]:
# Define the path
path = r'/Users/quincy/Documents/CF_Da/6/Airbnb Berlin/02 Data/Cleaned Data/'

# Join the path and filename
file_path0 = os.path.join(path, 'listing_cleaned.csv')
df_listing = pd.read_csv(file_path0)

In [61]:
# Aggregate the listings by neighborhood
neighbourhood_counts = df_listing.groupby('neighbourhood_group').size().reset_index(name='listing_count')

# Display the aggregated data
print(neighbourhood_counts.head())


        neighbourhood_group  listing_count
0      Charlottenburg-Wilm.            916
1  Friedrichshain-Kreuzberg           1688
2               Lichtenberg            228
3     Marzahn - Hellersdorf             97
4                     Mitte           1953


## Change mismatched spelling from df_listing to match the geojson file


In [62]:
# Dictionary for renaming mismatched neighborhoods
neighbourhood_rename_dict = {
    'Charlottenburg-Wilm.': 'Charlottenburg-Wilmersdorf',
    'Marzahn - Hellersdorf': 'Marzahn-Hellersdorf',
    'Steglitz - Zehlendorf': 'Steglitz-Zehlendorf',
    'Tempelhof - Schöneberg': 'Tempelhof-Schöneberg',
    'Treptow - Köpenick': 'Treptow-Köpenick'
}

# Rename the neighborhoods in df_listing
df_listing['neighbourhood_group'] = df_listing['neighbourhood_group'].replace(neighbourhood_rename_dict)

# Verify that the renaming was successful
print(df_listing['neighbourhood_group'].unique())


['Pankow' 'Friedrichshain-Kreuzberg' 'Neukölln' 'Mitte'
 'Charlottenburg-Wilmersdorf' 'Treptow-Köpenick' 'Tempelhof-Schöneberg'
 'Steglitz-Zehlendorf' 'Lichtenberg' 'Spandau' 'Reinickendorf'
 'Marzahn-Hellersdorf']


Check this worked

In [63]:
# Get unique values from both sources
geojson_neighbourhoods = set(city_geo['name'].unique())
df_neighbourhoods = set(df_listing['neighbourhood_group'].unique())

# Find mismatches again to ensure they are resolved
mismatched_neighbourhoods = geojson_neighbourhoods.symmetric_difference(df_neighbourhoods)
print("Mismatched neighbourhoods after renaming:", mismatched_neighbourhoods)


Mismatched neighbourhoods after renaming: set()


# Merge data

## Aggregate listings by neighbourhood

In [64]:
# Aggregate the listings by neighborhood
neighbourhood_counts = df_listing.groupby('neighbourhood_group').size().reset_index(name='listing_count')



In [65]:
# Merge the GeoJSON data with the listing counts
city_geo = city_geo.merge(neighbourhood_counts, left_on='name', right_on='neighbourhood_group', how='left')

In [66]:
#check this
city_geo.head()

,name,description,cartodb_id,created_at,updated_at,geometry,neighbourhood_group,listing_count
0,Mitte,,1,2013-09-03 12:32:04+02:00,2013-09-03 12:32:04+02:00,"MULTIPOLYGON (((1492073.912 6898398.347, 14921...",Mitte,1953
1,Friedrichshain-Kreuzberg,,2,2013-09-03 12:32:04+02:00,2013-09-03 12:32:04+02:00,"MULTIPOLYGON (((1494954.192 6892609.372, 14949...",Friedrichshain-Kreuzberg,1688
2,Pankow,,3,2013-09-03 12:32:04+02:00,2013-09-03 12:32:04+02:00,"MULTIPOLYGON (((1490488.054 6903530.308, 14903...",Pankow,1366
3,Charlottenburg-Wilmersdorf,,4,2013-09-03 12:32:04+02:00,2013-09-03 12:32:04+02:00,"MULTIPOLYGON (((1481972.781 6896952.359, 14819...",Charlottenburg-Wilmersdorf,916
4,Spandau,,5,2013-09-03 12:32:04+02:00,2013-09-03 12:32:04+02:00,"MULTIPOLYGON (((1463882.585 6903433.790, 14638...",Spandau,115


In [69]:
#it looks like there's timestamps which won't work with JSON so we need to get rid of those columns

In [70]:
# Drop the timestamp columns
city_geo_cleaned = city_geo.drop(columns=['created_at', 'updated_at'])

# Verify the columns have been dropped
print(city_geo_cleaned.dtypes)


name                     object
description              object
cartodb_id                int64
geometry               geometry
neighbourhood_group      object
listing_count             int64
dtype: object


In [83]:
# Drop the timestamp columns directly from city_geo
city_geo.drop(columns=['created_at', 'updated_at'], inplace=True)


# Create map of Airbnb Listing Concentration by neighbourhood

In [71]:
# Create a base map centered on Berlin
m = folium.Map(location=[52.52, 13.405], zoom_start=11)



In [72]:
# Create the choropleth map
Choropleth(
    geo_data=city_geo_cleaned,
    data=city_geo_cleaned,
    columns=['name', 'listing_count'],
    key_on='feature.properties.name',
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Number of Airbnb Listings per neighborhood'
).add_to(m)


In [73]:
# Save the map as an HTML file
map_path = r'/Users/quincy/Documents/CF_Da/6/Airbnb Berlin/04 Visualization/airbnb_choropleth.html'
m.save(map_path)
print(f"Choropleth map saved to {map_path}")

# Display the map
m

Choropleth map saved to /Users/quincy/Documents/CF_Da/6/Airbnb Berlin/04 Visualization/airbnb_choropleth.html


In [74]:
# Create a base map centered on Berlin
m = folium.Map(location=[52.52, 13.405], zoom_start=11)


## Aggregate data by pricing to create pricing neighbourhood map

In [76]:
df_listing.head()

,id,host_id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,neighb_type,review_scores_rating,accommodates,bathrooms,bedrooms,beds,amenities,host_is_superhost,host_response_rate,host_acceptance_rate,has_license,room_type2,short_long_stay
0,3176,3718,Pankow,Prenzlauer Berg Südwest,53,13,Entire home/apt,83,63,148,2023-05-25,1,1,112,1,2,5,4,1,1,2,"[""Hangers"", ""Crib"", ""Smoke alarm"", ""Fire extin...",f,NaN,0%,1,0,3
1,9991,33852,Pankow,Prenzlauer Berg Südwest,53,13,Entire home/apt,180,6,7,2020-01-04,0,1,17,0,2,5,7,2,4,7,"[""Hangers"", ""Smoke alarm"", ""Heating"", ""Luggage...",f,0%,0%,1,0,1
2,14325,55531,Pankow,Prenzlauer Berg Nordwest,53,13,Entire home/apt,70,150,26,2023-11-30,0,4,204,1,2,5,1,1,0,1,"[""Hangers"", ""Smoke alarm"", ""Heating"", ""Kitchen...",t,100%,36%,0,0,4
3,16644,64696,Friedrichshain-Kreuzberg,nördliche Luisenstadt,53,13,Entire home/apt,90,93,48,2017-12-14,0,2,230,0,0,5,4,1,1,1,"[""Hangers"", ""Carbon monoxide alarm"", ""Smoke al...",f,100%,50%,0,0,4
4,17904,68997,Neukölln,Reuterstraße,52,13,Entire home/apt,25,92,299,2022-12-01,2,1,1,0,3,5,2,1,0,1,"[""Hangers"", ""Self check-in"", ""Iron"", ""Heating""...",t,100%,98%,0,0,4


In [80]:
# Group by neighborhood and calculate the average price
neighborhood_prices = df_listing.groupby('neighbourhood_group')['price'].mean().reset_index()


In [82]:
# Create a base map centered on Berlin
m = folium.Map(location=[52.52, 13.405], zoom_start=11)

# Create the choropleth map with a pink and purple color palette
Choropleth(
    geo_data=city_geo_cleaned,
    data=neighborhood_prices,
    columns=['neighbourhood_group', 'price'],
    key_on='feature.properties.name',
    fill_color='BuPu',  # Use the 'BuPu' color palette for pink and purple shades
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Average Price per Night'
).add_to(m)

# Save the map as an HTML file
map_path = r'/Users/quincy/Documents/CF_Da/6/Airbnb Berlin/04 Visualization/average_price_map.html'
m.save(map_path)
print(f"Map saved to {map_path}")

# Display the map
m


Map saved to /Users/quincy/Documents/CF_Da/6/Airbnb Berlin/04 Visualization/average_price_map.html


In [88]:
# Define the file paths
df_listing_path = os.path.join(path, 'listing_cleaned2.csv')
city_geo_cleaned_path = os.path.join(path, 'city_geo_cleaned.csv')

# Export the datasets to CSV files
df_listing.to_csv(df_listing_path, index=False)
city_geo_cleaned.to_csv(city_geo_cleaned_path, index=False)


In [86]:
# Iterate over each row in df_listing and add a marker for each listing
for index, row in df_listing.iterrows():
    folium.Marker([row['latitude'], row['longitude']], popup=row['neighbourhood_group']).add_to(m)
